# ImageDataGenerator 

Image data generator for generating different types/angle of images from a single image

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [4]:
datagen = ImageDataGenerator(rotation_range = 40,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             fill_mode = 'nearest')

<li>
    rotation_width = degree of rotation
</li>
<li>
    width_shift_range = move the pic horizontally. 1 - move the pic completely at most. Less than one is prefered.
</li>
<li>
    height_shift_range = move the pic vertically. 1 - move the pic completely at most. Less than one is prefered.
</li>
<li>
    shear_range = pull the image on different sides
</li>
<li>
    zoom_range = zoom the image on sides different
</li>
<li>
    horizontal_flip = flip horizontal
</li>
<li>
    vertical_flip = flip vertically
</li>

##### Load the image

In [5]:
img = load_img ('/home/ghost/My_works/Malaria_Cell_Classification/cell_images/Parasitized/C99P60ThinF_IMG_20150918_142334_cell_17.png')

img.show()
x = img_to_array(img)

print(x.shape)

(226, 181, 3)


##### Resize the image using Antialiasing 

In [6]:
from PIL import Image

imResize = img.resize((180, 180), Image.ANTIALIAS)

imResize.show()

In [7]:
x = img_to_array(imResize)

In [8]:
print (x.shape)

(180, 180, 3)


##### Creating array of images from our dataset

In [18]:
import os

rootdir = '/home/ghost/My_works/Malaria_Cell_Classification/cell_images/Parasitized'

for subdir, dirs, files in os.walk(rootdir):
    i = 0
    img_list = []
    for file in files:
        #print(rootdir + file)
        if file.endswith('.png'):
            i = i + 1
            im = load_img(rootdir + '/' + file)
            im = im.resize((180, 180), Image.ANTIALIAS)
            i_arr = img_to_array(im)
            #print(i_arr.shape)
            i_arr = i_arr.reshape((1, ) + i_arr.shape)
            img_list.append(i_arr)
            im.save('Parasitized' + '/' + 'parasitized' + str(i) + '.png')
        
        if (i == 50):
            break

In [19]:
print(len(img_list))

50


In [23]:
import numpy as np

arr = np.array(img_list)

In [25]:
arr.shape

(50, 1, 180, 180, 3)

In [27]:
%%bash

mkdir preview

In [29]:
i = 0

##### Generate images  

In [30]:
for batch in datagen.flow(arr[1], batch_size=1,
                         save_to_dir='preview', save_prefix='el',
                         save_format = 'jpeg'):
    i += 1
    if i > 20:
        break
    

# Importing the keras classes 

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [32]:
model = Sequential()

In [34]:
model.add(Conv2D(32, (3, 3), input_shape=(180, 180, 3)))

In [35]:
model.add(Activation('relu'))

In [36]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [37]:
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [38]:
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [39]:
model.add(Flatten())

In [40]:
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [41]:
model.compile(loss= 'binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

In [42]:
batch_size = 16

train_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range= 0.2,
                zoom_range= 0.2,
                horizontal_flip=True)

In [44]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [49]:
train_generator = train_datagen.flow_from_directory('train',
                                                   target_size=(180, 180),
                                                   batch_size= batch_size,
                                                   class_mode='binary')

Found 288 images belonging to 2 classes.


In [50]:
validation_generator = test_datagen.flow_from_directory('validation',
                                                       target_size= (300, 300),
                                                       batch_size= batch_size,
                                                       class_mode='binary')

Found 108 images belonging to 2 classes.


In [52]:
import datetime

In [56]:
start_time = datetime.datetime.now()
print("Start time: " + str(start_time))
model.fit_generator(train_generator,
                   steps_per_epoch= 400,
                   epochs= 50,
                   validation_data= validation_generator,
                   validation_steps= 100)

model.save_weights('50_epochs.h5')
end_time = datetime.datetime.now()
print("End time: " + str(end_time))

print("Trained time: " + str(end_time-start_time))

Start time: 2019-07-27 00:52:43.973770
Epoch 1/50
399/400 [============================>.] - ETA: 0s - loss: 0.2356 - acc: 0.9345

ValueError: Error when checking input: expected conv2d_input to have shape (180, 180, 3) but got array with shape (300, 300, 3)